In [16]:
# Import dependencies
import pandas as pd
import os
import glob
import numpy as np
import datetime 

In [17]:
# Use glob to get all the supply data csv files in the current working directory
path = os.getcwd()
csv_files = glob.glob(os.path.join(path, "*.csv"))

In [18]:
# Create empty list
dataframes_list = []

In [19]:
# Append datasets to the list 
for i in csv_files:
    temp_df = pd.read_csv(i)
    dataframes_list.append(temp_df)

In [20]:
# Check dataframes_list length for all suppy data CSV files
len(dataframes_list)

1362

In [31]:
# Create empty list for transformed DataFrames
transformed_dfs = []

# Create empty DataFrame for all supply data
supply_df = pd.DataFrame()

In [32]:
# Transform each DataFrame in list
for df in dataframes_list:    
    # Create date variable
    date = df.columns[0]

    # Reset index to energy sources and transpose
    df_transposed = df.set_index(date).T
    
    # Create date column
    df_transposed["Date"] = date
    
    # Create time column 
    df_transposed["Time"] = df_transposed.index

    # Reset index to start from 0
    df_transposed.reset_index(drop=True, inplace=True)
    
    # Cast 'Time' column to string and append seconds
    df_transposed['Time'] = df_transposed['Time'].apply(str) + ':00'
    df_transposed.iloc[0:120, 9] = "0" + df_transposed.iloc[0:120, 9]
    
    # Add 'Date' and 'Time' columns together
    df_transposed['DateTime'] = df_transposed['Date'] + ' ' + df_transposed['Time']

    try:
        # Covert 'DateTime' column to datetime 
        df_transposed['DateTime'] =  pd.to_datetime(df_transposed['DateTime'])
    except:
        continue
        
    try:
        # Create 'Unix Timestamp' column
        df_transposed['Unix Timestamp'] = df_transposed.DateTime.apply(lambda x : (x-datetime.datetime(1970,1,1)).total_seconds())
    except:
        continue
    
    transformed_dfs.append(df_transposed)

In [33]:
transformed_dfs[650]

1/20/2020,Renewables,Natural gas,Large hydro,Imports,Batteries,Nuclear,Coal,Other,Date,Time,DateTime,Unix Timestamp
0,2024,8712,1188,6607,0,2275,10,0,1/20/2020,00:00:00,2020-01-20 00:00:00,1.579478e+09
1,2032,8660,1106,6696,0,2276,11,0,1/20/2020,00:05:00,2020-01-20 00:05:00,1.579479e+09
2,2015,8674,1035,6673,0,2277,11,0,1/20/2020,00:10:00,2020-01-20 00:10:00,1.579479e+09
3,2013,8798,1006,6598,-10,2276,10,0,1/20/2020,00:15:00,2020-01-20 00:15:00,1.579479e+09
4,2007,8635,990,6709,0,2275,10,0,1/20/2020,00:20:00,2020-01-20 00:20:00,1.579480e+09
...,...,...,...,...,...,...,...,...,...,...,...,...
283,2768,8754,1289,7422,-74,2273,10,0,1/20/2020,23:35:00,2020-01-20 23:35:00,1.579563e+09
284,2776,8506,1361,7376,1,2273,11,0,1/20/2020,23:40:00,2020-01-20 23:40:00,1.579564e+09
285,2803,8275,1430,7334,0,2273,11,0,1/20/2020,23:45:00,2020-01-20 23:45:00,1.579564e+09
286,2842,8154,1445,7331,-37,2274,11,0,1/20/2020,23:50:00,2020-01-20 23:50:00,1.579564e+09


In [ ]:
#for df in transformed_dfs:
    #supply_df = supply_df.append(df, ignore_index = True)
    #print(supply_df.head())

# Append to master DataFrame with all supply data
#supply_df = supply_df.append(dataframes_list, ignore_index = True) 
#supply_df.head()
    
# Export supply_df to CSV